In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

In [166]:
listings=pd.read_csv("C://Users//Harish//Documents//Autumn Quarter Assignments//BigData Analytics//listings (1).csv")
reviews=pd.read_csv("C://Users//Harish//Documents//Autumn Quarter Assignments//BigData Analytics//reviews (1).csv")

In [167]:
listings.drop(['name','host_id','host_name','latitude','longitude','minimum_nights','neighbourhood_group'
            ,'calculated_host_listings_count','reviews_per_month','number_of_reviews']
            ,inplace=True, axis=1)
listings['neighbourhood'] = listings['neighbourhood'].str.strip()
listings['room_type'] = listings['room_type'].str.strip()


In [168]:
listings = listings[listings.price != 0]
listings = listings[listings.last_review >= '2017-01-01']
reviews = reviews[reviews.date >= '2017-01-01']

In [169]:
reviews = reviews.groupby('listing_id')['date'].size().reset_index(name='number_of_reviews')
#reviews = reviews.groupby('listing_id')[pd.Grouper(freq='M')].mean().reset_index(name='number_of_reviews per month')
listings = listings.rename(columns={"id": "listing_id"})
listings = pd.merge(listings, reviews, how='inner', on='listing_id')


In [170]:
listings=listings.drop(columns='last_review',axis=1)



In [171]:
X=listings.loc[:,['neighbourhood','room_type','availability_365','number_of_reviews']].values
y=listings.loc[:,'price'].values

print(X)
print(y)




[['Madrona' 'Entire home/apt' 322 18]
 ['Roosevelt' 'Private room' 349 12]
 ['South Delridge' 'Entire home/apt' 14 98]
 ...
 ['Broadway' 'Entire home/apt' 355 1]
 ['Belltown' 'Private room' 3 1]
 ['Brighton' 'Entire home/apt' 11 1]]
[296  82  48 ... 120  65 120]


In [172]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder = LabelEncoder()
X[:,0]=labelencoder.fit_transform(X[:,0])
X[:,1]=labelencoder.fit_transform(X[:,1])
onehotencoder = OneHotEncoder(categorical_features=[0,1])
X=onehotencoder.fit_transform(X).toarray()


In [173]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(y_train.shape)

(5721,)


In [174]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [175]:
y_pred=regressor.predict(X_test)


In [162]:
analysis=pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
analysis.head()
# y_pred=pd.DataFrame(y_pred)
# y_pred.to_csv('predictions.csv')
# y_test=pd.DataFrame(y_test)
# y_test.to_csv('actual.csv')

,Actual,Predicted
0,190,141.470292
1,48,73.358510
2,142,212.763723
3,64,36.564592
4,400,189.330964


In [154]:
# import statsmodels.formula.api as sm
# X=np.append(arr=np.ones((7152,1)).astype(int),values=X,axis=1)
# x_opt=X
# regressor_OLS=sm.OLS(endog=y,exog=x_opt).fit()
# regressor_OLS.summary()

In [155]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
# #vif["features"] = X.columns

In [176]:
# plt.scatter(X_test[:,-1],y_test,color='red')
# plt.plot(X_train[:,-1],regressor.predict(X_train[:,-1]),color='blue')
# plt.show()

In [177]:
from sklearn import metrics
mse=metrics.mean_squared_error(y_test,y_pred)
print(mse)
rmse=np.sqrt(mse)
print(rmse)

6754.000525715206
82.18272644367067
